In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from fogo_cruzado_api import FogoCruzadoApi, Credentials

In [2]:
load_dotenv()

True

In [3]:
credentials = {
    "email": os.getenv("FOGO_CRUZADO_API_EMAIL"),
    "password": os.getenv("FOGO_CRUZADO_API_PASSWORD"),
}

In [22]:
fogo = FogoCruzadoApi(
    credentials=Credentials(email=credentials["email"], password=credentials["password"]))
occurrences_data = fogo.get_all_occurrences("b112ffbe-17b3-4ad0-8f2a-2038745d1d14", '2023-01-01', '2024-01-01')
print(len(occurrences_data))
# print(occurrences_data[0])

400
{"msg":"Bad Request","msgCode":"validation_error","code":400,"errors":["Id do Estado, deve ser um UUID"]}
0


In [5]:
data = {
    'id': 'bff14f2b-f468-46d9-a82d-3ea2cb8e0011',
    'documentNumber': 5325,
  'address': 'Cidade de Deus, Rio de Janeiro - RJ, Brasil',
  'state': {'id': 'b112ffbe-17b3-4ad0-8f2a-2038745d1d14',
   'name': 'Rio de Janeiro'},
  'region': {'id': '31d96bb1-07fa-4a9e-b988-9b1beae828ed',
   'region': 'Sudeste',
   'state': 'Rio de Janeiro',
   'enabled': True},
  'city': {'id': 'd1bf56cc-6d85-4e6a-a5f5-0ab3f4074be3',
   'name': 'RIO DE JANEIRO'},
  'neighborhood': {'id': '179f5e50-ff62-48aa-b42c-fa9368cd6ca6',
   'name': 'CIDADE DE DEUS'},
  'subNeighborhood': None,
  'locality': {'id': '1fde637d-57a9-4018-aed5-7f788e030447',
   'name': 'CIDADE DE DEUS'},
  'latitude': '-22.9481697083',
  'longitude': '-43.3626708984',
  'date': '2016-07-05T00:00:00.000Z',
  'policeAction': True,
  'agentPresence': True,
  'relatedRecord': None,
  'contextInfo': {'mainReason': {'id': '86b1d37b-f955-4015-9d63-3c210a845e5e',
    'name': 'Ação policial'},
   'complementaryReasons': [],
   'clippings': [],
   'massacre': False,
   'policeUnit': 'N�o identificado'},
  'transports': [],
  'victims': [],
  'animalVictims': []}

In [15]:
# data = list(map(lambda occur: occur.get("locality", {}).get("name"), occurrences_data))

data = []
for occur in occurrences_data:    
    try:
        locality = occur.get("locality", {})
        if locality: 
            locality = locality.get("name", None)
        
        neighborhood = occur.get("neighborhood", {})
        if neighborhood:
            neighborhood = neighborhood.get("name", None)
            
        sub_neighborhood = occur.get("subNeighborhood", {})
        if sub_neighborhood:
            sub_neighborhood = sub_neighborhood.get("name", None)
            
        context_info = occur.get("contextInfo", {})
        if context_info:
            reason = context_info.get("mainReason", {})
            if reason:
                reason = reason.get("name", None)
            
            massacre = context_info.get("massacre", False)
    
        data.append({
            'id': occur.get("id", None),
            "city": occur.get("city", {}).get("name", None),
            "locality": locality,
            "neighborhood": neighborhood,
            "sub_neighborhood": sub_neighborhood,
            "latitude": occur.get("latitude", None),
            "longitude": occur.get("longitude", None),
            "date": occur.get("date", None),
            "reason": reason,
            "massacre": massacre,
            "police_presence": occur.get("policeAction", None),
            "security_agent_presence": occur.get("agentPresence"),
            "victims": occur.get("victims", [])
        })
    except TypeError as e:
        print("Occurrence empty")

print(data)
print(len(data))

[{'id': '00a70fa3-572f-46b4-8313-23849350ff13', 'city': 'RIO DE JANEIRO', 'locality': None, 'neighborhood': 'BANGU', 'sub_neighborhood': None, 'latitude': '-22.8742263', 'longitude': '-43.4685442', 'date': '2023-01-01T00:02:00.000Z', 'reason': 'Tiros a esmo', 'massacre': False, 'policePresence': False, 'securityAgentPresence': False, 'victims': []}, {'id': '05b43438-a189-4f9d-8935-1d37c1d33a69', 'city': 'RIO DE JANEIRO', 'locality': None, 'neighborhood': 'BOTAFOGO', 'sub_neighborhood': None, 'latitude': '-22.9543423', 'longitude': '-43.1955827', 'date': '2023-01-01T00:07:00.000Z', 'reason': 'Não identificado', 'massacre': False, 'policePresence': False, 'securityAgentPresence': False, 'victims': [{'id': 'c0addabf-d7bd-463e-9465-3062b1cbdd11', 'occurrenceId': '05b43438-a189-4f9d-8935-1d37c1d33a69', 'type': 'People', 'situation': 'Wounded', 'circumstances': [{'id': 'b44d4f46-e7e4-4822-a42e-660927796731', 'name': 'Bala perdida', 'type': 'both'}], 'deathDate': None, 'personType': 'Civilian

In [16]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)

In [18]:
df = pd.DataFrame(data, columns=[
    "id", 
    "city", 
    "locality", 
    "neighborhood", 
    "sub_neighborhood", 
    "latitude", "longitude", 
    "date", "reason", 
    "massacre", "police_presence", "security_agent_presence", 
    "victims"])
df

,id,city,locality,neighborhood,sub_neighborhood,latitude,longitude,date,reason,massacre,police_presence,security_agent_presence,victims
0,00a70fa3-572f-46b4-8313-23849350ff13,RIO DE JANEIRO,None,BANGU,None,-22.8742263,-43.4685442,2023-01-01T00:02:00.000Z,Tiros a esmo,False,NaN,NaN,[]
1,05b43438-a189-4f9d-8935-1d37c1d33a69,RIO DE JANEIRO,None,BOTAFOGO,None,-22.9543423,-43.1955827,2023-01-01T00:07:00.000Z,Não identificado,False,NaN,NaN,[{'id': 'c0addabf-d7bd-463e-9465-3062b1cbdd11'...
2,9a972999-ed8c-4cfc-8b9f-49ff3aa6e02d,RIO DE JANEIRO,None,RECREIO DOS BANDEIRANTES,None,-23.0174785,-43.4625904,2023-01-01T00:07:00.000Z,Tiros a esmo,False,NaN,NaN,[]
3,c86c1ec6-d02f-4c7c-b29d-aa4b85496924,RIO DE JANEIRO,SANTO AMARO,CATETE,None,-22.9236111,-43.1805556,2023-01-01T00:09:00.000Z,Tiros a esmo,False,NaN,NaN,[]
4,e7144386-75c3-413b-b39e-ce7e4b5857a6,MESQUITA,None,SANTA TEREZINHA,None,-22.7906338,-43.4394281,2023-01-01T00:10:00.000Z,Não identificado,False,NaN,NaN,[{'id': 'd4d03366-0a70-4358-9bda-eaeea6776754'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,9f229472-8748-4688-8247-416fe391aa22,RIO DE JANEIRO,None,JACARE,None,-22.8923272,-43.2581953,2024-01-01T00:49:00.000Z,Não identificado,False,NaN,NaN,[]
2956,06c36411-7768-4d53-abf7-11fe1905035d,SÃO JOÃO DE MERITI,None,COELHO DA ROCHA,None,-22.7768834,-43.3849625,2024-01-01T03:00:00.000Z,Não identificado,False,NaN,NaN,[{'id': '4c8d36f6-50e5-4c07-ae52-bdfe787f10eb'...
2957,456d1919-5198-4dc8-9d9d-b07a82c798d0,SÃO JOÃO DE MERITI,None,VILAR DOS TELES,JARDIM BOTANICO,-22.77487137897685,-43.35501073314844,2024-01-01T04:15:00.000Z,Não identificado,False,NaN,NaN,[]
2958,b534d773-d826-4e8e-9f42-afa0eeabc21b,RIO DE JANEIRO,None,INHOAIBA,VILA SAO JORGE,-22.912557450895576,-43.58366704536351,2024-01-01T19:15:00.000Z,Não identificado,False,NaN,NaN,[]


In [19]:
fogo.get_states()

[{'id': 'b112ffbe-17b3-4ad0-8f2a-2038745d1d14', 'name': 'Rio de Janeiro'},
 {'id': '813ca36b-91e3-4a18-b408-60b27a1942ef', 'name': 'Pernambuco'},
 {'id': 'd3a9b545-7056-4dc6-9b68-ce320c9edffc', 'name': 'Bahia'},
 {'id': '2a98a020-3815-45d7-a6f6-6de2119eba8b', 'name': 'Pará'}]